# Denoising Methods - Filters

In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np
from PIL import Image

from image_processing_utilities.functions import l2_samples, ssim_samples, psnr_samples, dataset_generator

In [2]:
dataset = 'SIDD'
method = 'Gaussian'

val_noisy_mat, val_gt_mat = dataset_generator()

In [ ]:
# Sample Images
image_samples = [1, 10, 17, 23]
image_crops = [2, 4, 7, 11]
samples_noisy = np.array([[val_noisy_mat[i, j] for j in image_crops] for i in image_samples])
samples_gt = np.array([[val_gt_mat[i, j] for j in image_crops] for i in image_samples])

In [ ]:
# Create subplot figure with 2 rows and 2 columns
fig = make_subplots(rows=4, cols=4, subplot_titles=(
    "Noisy 01", "GT 01", "Noisy 02", "GT 02", 
    "Noisy 03", "GT 03", "Noisy 04", "GT 04",
    "Noisy 05", "GT 05", "Noisy 06", "GT 06", 
    "Noisy 07", "GT 07", "Noisy 08", "GT 08"
))

# Add images to the subplots
fig.add_trace(go.Image(z=samples_noisy[0][1]), row=1, col=1)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[0][1]), row=1, col=2)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[1][0]), row=1, col=3)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[1][0]), row=1, col=4)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[1][3]), row=2, col=1)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[1][3]), row=2, col=2)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[2][0]), row=2, col=3)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[2][0]), row=2, col=4)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][0]), row=3, col=1)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[3][0]), row=3, col=2)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][1]), row=3, col=3)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[3][1]), row=3, col=4)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][2]), row=4, col=1)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[3][2]), row=4, col=2)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][3]), row=4, col=3)  # Noisy Image
fig.add_trace(go.Image(z=samples_gt[3][3]), row=4, col=4)  # Ground Truth

# Remove axes for all subplots
fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)

fig.update_layout(height=1000, 
                  width=1200, 
                  title_text="Noisy and Ground Truth Images",
                  margin=dict(l=20, r=20, t=80, b=20),  # Reduce margins
)
fig.show()

# Gaussian Filter

In [ ]:
print('ORIGINAL VALUES\n')

print('L2: ', l2_samples(samples_noisy, samples_gt))
print('SSIM: ', ssim_samples(samples_noisy, samples_gt))
print('PSNR: ', psnr_samples(samples_noisy, samples_gt))

In [ ]:
# Grid search find the best parameters:
kernels = list(range(1, 51, 2))
sigmas = list(np.arange(1, 15.25, .25))

In [ ]:
best_loss = 1e4
best_kernel = None
best_sigma = None
best_index = None

metrics = np.zeros((len(kernels), len(sigmas)))
for i, kernel in enumerate(kernels):
    for j, sigma in enumerate(sigmas):
        test = gaussian_blurr_samples(samples_noisy, kernel, sigma)
        
        avg_loss = 1 - ssim_samples(test, samples_gt)
        metrics[i, j] = avg_loss

        if avg_loss < best_loss:
            best_index = [i, j]
            best_kernel = kernel
            best_sigma = sigma
            best_loss = avg_loss
    
print(best_kernel, best_sigma)

In [ ]:
metrics_normalized = (metrics - metrics.min()) / (metrics.max() - metrics.min())

fig = go.Figure(data=go.Heatmap(z=metrics_normalized.T, x=kernels, y=sigmas, colorscale='viridis'))
# Add a star annotation for the minimum value
fig.add_annotation(
    x=best_kernel, y=best_sigma,
    text="★ min",
    showarrow=False,
    font=dict(size=10, color="red")
)    
fig.update_layout(height=500, width=1000, 
    xaxis_title='Kernels',
    yaxis_title='Sigmas'
)

fig.show()


In [ ]:
# Create the 3D surface plot
fig = go.Figure(data=[go.Surface(z=metrics_normalized.T, x=kernels, y=sigmas, colorscale='Viridis')])

# Update layout
fig.update_layout(height=500, width=600,
    scene=dict(
        xaxis_title='Kernels',
        yaxis_title='Sigmas',
        zaxis_title='Normalized Loss'
    )
)

# Show the plot
fig.show()

In [ ]:
denoised = gaussian_blurr_samples(samples_noisy, best_kernel, best_sigma)

print('Denoised VALUES\n')

print('L2: ', l2_samples(denoised, samples_gt))
print('SSIM: ', ssim_samples(denoised, samples_gt))
print('PSNR: ', psnr_samples(denoised, samples_gt))

In [ ]:
# Create subplot figure with 2 rows and 2 columns
fig = make_subplots(rows=4, cols=6, subplot_titles=(
    "Noisy 01", "Denoised 01", "GT 01", "Noisy 02", "Denoised 02", "GT 02", 
    "Noisy 03", "Denoised 03", "GT 03", "Noisy 04", "Denoised 04", "GT 04",
    "Noisy 05", "Denoised 05", "GT 05", "Noisy 06", "Denoised 06", "GT 06", 
    "Noisy 07", "Denoised 07", "GT 07", "Noisy 08", "Denoised 08", "GT 08"
))

# Add images to the subplots
fig.add_trace(go.Image(z=samples_noisy[0][1]), row=1, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised[0][1]), row=1, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[0][1]), row=1, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[1][0]), row=1, col=4)  # Noisy Image
fig.add_trace(go.Image(z=denoised[1][0]), row=1, col=5)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[1][0]), row=1, col=6)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[1][3]), row=2, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised[1][3]), row=2, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[1][3]), row=2, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[2][0]), row=2, col=4)  # Noisy Image
fig.add_trace(go.Image(z=denoised[2][0]), row=2, col=5)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[2][0]), row=2, col=6)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][0]), row=3, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised[3][0]), row=3, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[3][0]), row=3, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][1]), row=3, col=4)  # Noisy Image
fig.add_trace(go.Image(z=denoised[3][1]), row=3, col=5)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[3][1]), row=3, col=6)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][2]), row=4, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised[3][2]), row=4, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[3][2]), row=4, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][3]), row=4, col=4)  # Noisy Image
fig.add_trace(go.Image(z=denoised[3][3]), row=4, col=5)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[3][3]), row=4, col=6)  # Ground Truth

# Remove axes for all subplots
fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)

fig.update_layout(height=1000, 
                  width=1200, 
                  # title_text="Noisy and Ground Truth Images",
                  margin=dict(l=40, r=40, t=40, b=20),  # Reduce margins
)
fig.show()

In [ ]:
# Create subplot figure with 2 rows and 2 columns
fig = make_subplots(rows=3, cols=3, subplot_titles=(
    "Noisy 01", "Denoised 01", "GT 01", "Noisy 02", "Denoised 02", "GT 02", 
    "Noisy 03", "Denoised 03", "GT 03"
))

# Add images to the subplots
fig.add_trace(go.Image(z=samples_noisy[0][1]), row=1, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised[0][1]), row=1, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[0][1]), row=1, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[1][3]), row=2, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised[1][3]), row=2, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[1][3]), row=2, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[2][0]), row=3, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised[2][0]), row=3, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[2][0]), row=3, col=3)  # Ground Truth

# Remove axes for all subplots
fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)

fig.update_layout(height=500, 
                  width=1000, 
                  # title_text="Noisy and Ground Truth Images",
                  margin=dict(l=20, r=20, t=20, b=20),  # Reduce margins
)
fig.show()

In [ ]:
# Convert the NumPy array to a PIL Image object
img = Image.fromarray(samples_gt[2][0])

# Save the image
# img.save("gt_3.png")